# Capstone 2

## Polish Bankruptcy

## Pre-processing and Training

In the pre-processing step I will impute the missing values and from the data, drop redundent high correlation variables from the dataset, complete a stratified train test split and and investigate scaling the training data.  Because of the high amount of skewed data the possibility of transforming the data may need to be done after the initial models.  

## Import Statements

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from random import randint, choice
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Load Data

In [2]:
#load bankruptcy data and column key data
bankruptcy_data = pd.read_csv('bankruptcy_data_comb.csv')
data_columns = pd.read_csv('column_key.csv')

#load high correlation dataframe
dataCorrhigh = pd.read_csv('dataCorrhigh.csv')

#display high correlation pairs dataframe
print(dataCorrhigh.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Feature1     111 non-null    object 
 1   Feature2     111 non-null    object 
 2   Correlation  111 non-null    float64
dtypes: float64(1), object(2)
memory usage: 2.7+ KB
None


## Impute missing data

Based of the distribution of data and the number of outliers, I have made the decsion to imput the missing values based on the median of each column.

In [3]:
#create new dataframe by imputing missing values with the median
bankruptcy_complete = bankruptcy_data.apply(lambda x: x.fillna(x.median()),axis=0)

#display complete dataframe
bankruptcy_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43405 entries, 0 to 43404
Data columns (total 66 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X01     43405 non-null  float64
 1   X02     43405 non-null  float64
 2   X03     43405 non-null  float64
 3   X04     43405 non-null  float64
 4   X05     43405 non-null  float64
 5   X06     43405 non-null  float64
 6   X07     43405 non-null  float64
 7   X08     43405 non-null  float64
 8   X09     43405 non-null  float64
 9   X10     43405 non-null  float64
 10  X11     43405 non-null  float64
 11  X12     43405 non-null  float64
 12  X13     43405 non-null  float64
 13  X14     43405 non-null  float64
 14  X15     43405 non-null  float64
 15  X16     43405 non-null  float64
 16  X17     43405 non-null  float64
 17  X18     43405 non-null  float64
 18  X19     43405 non-null  float64
 19  X20     43405 non-null  float64
 20  X21     43405 non-null  float64
 21  X22     43405 non-null  float64
 22

## Drop redundant high correlation  columns


In [4]:
# drop first row of correlation dataframe with correlation of 1.  Data is identical 
dataCorrhigh = dataCorrhigh.drop(dataCorrhigh.index[0]).reset_index(drop=True)

#create empty final high correlation dataframe
corr_high_final = []

# remove redundant feature X07
dataCorrhigh = dataCorrhigh.drop(dataCorrhigh[(dataCorrhigh['Feature1'] == 'X07') | (dataCorrhigh['Feature2'] == 'X07')].index,axis = 0 ,inplace = False)
dataCorrhigh = dataCorrhigh.reset_index(drop=True)

# iterate over high correlation dataframe and use random number to determine final feture dataframe
for x in range(len(dataCorrhigh)):
    k = random.randint(0, 1)
    if k == 0:
        corr_high_final.append(dataCorrhigh['Feature1'][x])
        
    else:
        corr_high_final.append(dataCorrhigh['Feature2'][x])
   
print(corr_high_final)

['X58', 'X46', 'X58', 'X56', 'X17', 'X23', 'X31', 'X43', 'X38', 'X53', 'X23', 'X51', 'X50', 'X50', 'X14', 'X36', 'X56', 'X43', 'X43', 'X16', 'X56', 'X58', 'X44', 'X33', 'X22', 'X16', 'X22', 'X26', 'X44', 'X43', 'X20', 'X56', 'X49', 'X48', 'X63', 'X49', 'X32', 'X34', 'X47', 'X63', 'X63', 'X02', 'X51', 'X62', 'X54', 'X64', 'X44', 'X30', 'X22', 'X11', 'X33', 'X22', 'X35', 'X62', 'X09', 'X11', 'X14', 'X56', 'X30', 'X20', 'X63', 'X35', 'X11', 'X22', 'X33', 'X35', 'X33', 'X18', 'X62', 'X43', 'X48', 'X48', 'X20', 'X56', 'X58', 'X48', 'X25', 'X38', 'X06', 'X34', 'X36', 'X35', 'X35', 'X11', 'X24', 'X06', 'X51', 'X24', 'X24', 'X24', 'X34', 'X24', 'X13', 'X13', 'X20', 'X11', 'X34', 'X35', 'X43', 'X63', 'X44', 'X13', 'X18']


## Create final feature array 

In [5]:
# get unique feature array
feature_df = np.unique(corr_high_final)
print(feature_df)

['X02' 'X06' 'X09' 'X11' 'X13' 'X14' 'X16' 'X17' 'X18' 'X20' 'X22' 'X23'
 'X24' 'X25' 'X26' 'X30' 'X31' 'X32' 'X33' 'X34' 'X35' 'X36' 'X38' 'X43'
 'X44' 'X46' 'X47' 'X48' 'X49' 'X50' 'X51' 'X53' 'X54' 'X56' 'X58' 'X62'
 'X63' 'X64']


## Remove non correlated columns from bankruptcy data

In [6]:
#create new dataframe with high correlation features
bankruptcy_data_final = bankruptcy_complete[feature_df].copy()

# add back in the year and class features
bankruptcy_data_final = pd.concat([bankruptcy_data_final,bankruptcy_complete['Year']], axis = 1)
bankruptcy_data_final = pd.concat([bankruptcy_data_final,bankruptcy_complete['Class']], axis = 1)
print(bankruptcy_data_final.head(20))

         X02       X06      X09       X11       X13       X14        X16  \
0   0.379510  0.388250  1.13890  0.249760  0.166600  0.249760   0.733780   
1   0.499880  0.000000  1.69960  0.261140  0.158350  0.258340   0.538380   
2   0.695920  0.000000  1.30900  0.312580  0.244350  0.309060   0.459610   
3   0.307340  0.149880  1.05710  0.092704  0.094257  0.092704   0.398030   
4   0.613230  0.187320  1.15590  0.187320  0.121820  0.187320   0.322110   
5   0.497940  0.000000  1.97860  0.286450  0.148120  0.281390   0.588580   
6   0.647440  0.000000  1.73480  0.125750  0.309630  0.111090   0.829650   
7   0.027059  0.000000  0.65273  0.693940  1.060200  0.652400  25.575000   
8   0.632020  0.000000  1.33320  0.043162  0.038938  0.014434   0.082138   
9   0.838370  0.000000  2.11560  0.184540  0.075411  0.153280   0.190300   
10  0.443550 -0.931900  4.74470  0.242270  0.195100  0.240010   2.087000   
11  0.111480 -0.084883  0.90732 -0.024300  0.101640 -0.024300   0.483470   
12  0.349940

## Train test split


Train test split will be done with the stardard 75/25 split and the data stratified by bankruptcy class to maintain ratio of bankrupt and non-bankrupt entries.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(bankruptcy_data_final.drop(columns='Class'), bankruptcy_data_final.Class,
                                                    test_size=.25  ,random_state=5, stratify=bankruptcy_data_final.Class)

print(X_train)

           X02       X06      X09       X11       X13       X14       X16  \
12113  1.39110 -1.630900  0.88906 -1.068200 -0.104620 -1.068200 -0.147250   
3289   0.25636  0.035034  0.98773  0.014925  0.023540  0.014925  0.256750   
16567  0.54284  0.188430  1.00990  0.013167  0.032714  0.013167  0.083351   
21367  0.54860  0.297050  1.12700  0.119670  0.125330  0.119670  0.267740   
32394  0.10553  0.390000  1.16000  0.152710  0.196260  0.152710  1.864000   
...        ...       ...      ...       ...       ...       ...       ...   
24299  2.15570 -0.072920  1.14410 -1.080300 -0.961970 -1.138900 -0.510570   
41350  0.43133  0.217130  1.07930  0.143070  0.092277  0.143070  0.386680   
38703  0.55160  0.000000  1.18680  0.137730  0.093009  0.079772  0.200110   
12352  0.15038 -0.000078  0.99346  0.152910  0.194080  0.151780  1.282100   
24962  0.69258 -0.054472  6.12300  0.051654  0.016724  0.032481  0.147860   

           X17       X18      X20  ...      X50      X51      X53      X54 

In [8]:
#display X training and test sizes
X_train.shape, X_test.shape

((32553, 39), (10852, 39))

In [9]:
# display y training and test sizes
y_train.shape, y_test.shape

((32553,), (10852,))

In [10]:
# verify proper training columns
print(X_train.columns)

Index(['X02', 'X06', 'X09', 'X11', 'X13', 'X14', 'X16', 'X17', 'X18', 'X20',
       'X22', 'X23', 'X24', 'X25', 'X26', 'X30', 'X31', 'X32', 'X33', 'X34',
       'X35', 'X36', 'X38', 'X43', 'X44', 'X46', 'X47', 'X48', 'X49', 'X50',
       'X51', 'X53', 'X54', 'X56', 'X58', 'X62', 'X63', 'X64', 'Year'],
      dtype='object')


## Scale the data

Based on the range of the data and the degree of outliers the data was scaled before the logistic regression.

In [11]:
# scale fit and transform the data using standard scaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Create the Logistic Regression object 

In [12]:
# create the regression object
clf = LogisticRegression(max_iter=1000)
# fit the scaled and training data
clf.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [13]:
# run the regression model
y_te_pred = clf.predict(X_test_scaled)
y_predict_training = clf.predict(X_train_scaled)

## Accuracy Scores

In [14]:
# print test accuracy
print("\n")
print("[Test] Accuracy score: (y_test, y_te_pred)",accuracy_score(y_test, y_te_pred))

# print training accuracy
print("\n")
print("[Training] Accuracy score: (y_train, y_predict_training)",accuracy_score(y_train, y_predict_training))



[Test] Accuracy score: (y_test, y_te_pred) 0.951529671949871


[Training] Accuracy score: (y_train, y_predict_training) 0.9518631155346665


## Classification Report

In [15]:
# print classification scores  0 is non bankrupt entities 1 is bankrupt entitiies
print("[Training Classification Report]")
print(classification_report(y_train, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_te_pred))

[Training Classification Report]
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     30985
           1       0.52      0.01      0.02      1568

    accuracy                           0.95     32553
   macro avg       0.74      0.50      0.50     32553
weighted avg       0.93      0.95      0.93     32553

[Test Classification Report]
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     10329
           1       0.36      0.01      0.01       523

    accuracy                           0.95     10852
   macro avg       0.66      0.50      0.50     10852
weighted avg       0.92      0.95      0.93     10852



The overll accuracy score is .95 due to the the size of the non bankrupt entities.  The logistic regression model predicts the non bankrupt entities very well but the does not do as well with the banrkrupt entities at .52 for the training data and fares worse on the test data at .36.

## Summary and further steps


In the pre-processing step, I imputed the missing values, removed redundant correlation pairs and created the final feature dataframe.  Then I created the train test split, scaled the data and trained the logistic regression model.  Finally I checked the accuracy scores and classification report.  In the modeling step I will explore other models, evaluate performance metrics and identify the best performing model.     